## 爬链家各个房间的网站

In [1]:
import re
import requests
from urllib.request import Request, urlopen
from selenium import webdriver
from lxml import etree
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup 
import time

In [2]:
def get_lianjia_info(page):   #获取每个房源的link以及小区位置信息
    url = 'https://sh.lianjia.com/zufang/pg' + str(page)
    headers = {
            'Referer':'https://sh.lianjia.com/zufang/',
                 'Upgrade-Insecure-Requests':'1',
                 'User-Agent':'Mozilla/5.0(WindowsNT6.3;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/68.0.3440.106Safari/537.36'}
    html = requests.get(url,headers=headers)
    selector = etree.HTML(html.content)

    web = selector.xpath('//*[@class="content__list--item--title"]/a/@href')
    title = selector.xpath('//*[@class="content__list--item--title"]/a/text()')
    districts = selector.xpath('//*[@class="content__list--item--des"]/a[1]/text()')
    street = selector.xpath('//*[@class="content__list--item--des"]/a[2]/text()')
    estate = selector.xpath('//*[@class="content__list--item--des"]/a[3]/text()')
    
    districts=pd.DataFrame(districts)
    street=pd.DataFrame(street)
    estate=pd.DataFrame(estate)
    title=pd.DataFrame(title)

    web=pd.DataFrame(web)
    web = pd.concat([web,title,districts,street,estate],axis=1)
    
    web.columns=['网站','名称','区','街道','小区']
    web['网站']='http://sh.lianjia.com'+web['网站']
    return web


In [3]:
web=get_lianjia_info(0)
web['网站'][0]

'http://sh.lianjia.com/zufang/SH2781717703893983232.html'

## 爬取多页

In [6]:
web=pd.DataFrame(columns=['网站','名称','区','街道','小区'])
s_time = time.time()
for i in range(0,10):       #爬取前10页  链家上有100页 每页30个   
    web=web.append(get_lianjia_info(i),ignore_index=True)
    #time.sleep(1)
    print('第'+str(i)+'页爬取完成')
print(web['网站'][0])
e_time = time.time()
print('the program runs for {} seconds'.format(e_time-s_time))


第0页爬取完成
第1页爬取完成
第2页爬取完成
第3页爬取完成
第4页爬取完成
第5页爬取完成
第6页爬取完成
第7页爬取完成
第8页爬取完成
第9页爬取完成
http://sh.lianjia.com/zufang/SH2781717703893983232.html
the program runs for 4.540961027145386 seconds


In [7]:
web

,网站,名称,区,街道,小区
0,http://sh.lianjia.com/zufang/SH278171770389398...,\n 整租·双秀西园 2室2厅 南/北,浦东,康桥,双秀西园
1,http://sh.lianjia.com/zufang/SH278199691817412...,\n 整租·高兴花园 1室1厅 南,闵行,春申,高兴花园
2,http://sh.lianjia.com/zufang/SH216362156310136...,\n 整租·翠庭 2室2厅 南,长宁,虹桥,翠庭
3,http://sh.lianjia.com/zufang/SH224623160808257...,\n 整租·古北嘉年华庭 3室2厅 南,长宁,古北,古北嘉年华庭
4,http://sh.lianjia.com/zufang/SH201375709751857...,\n 整租·西郊华城 5室3厅 复式 南,青浦,徐泾,西郊华城
...,...,...,...,...,...
295,http://sh.lianjia.com/zufang/SH273995825793336...,\n 整租·沿海丽水馨庭(别墅) 5室2厅 南,松江,莘闵别墅,沿海丽水馨庭(别墅)
296,http://sh.lianjia.com/zufang/SH274193324412686...,\n 整租·高兴花园 2室1厅 南,闵行,春申,高兴花园
297,http://sh.lianjia.com/zufang/SH274066724754581...,\n 整租·合生江湾国际公寓 2室2厅 南,杨浦,新江湾城,合生江湾国际公寓
298,http://sh.lianjia.com/zufang/SH274134492568906...,\n 整租·东鼎名人府邸(松江) 2室1厅 南,松江,松江新城,东鼎名人府邸(松江)


In [8]:
web.to_csv('web.csv',index=False,encoding='utf-8-sig')

##  爬取各个房间信息

In [9]:
web=pd.read_csv('web.csv')
web

,网站,名称,区,街道,小区
0,http://sh.lianjia.com/zufang/SH278171770389398...,\n 整租·双秀西园 2室2厅 南/北,浦东,康桥,双秀西园
1,http://sh.lianjia.com/zufang/SH278199691817412...,\n 整租·高兴花园 1室1厅 南,闵行,春申,高兴花园
2,http://sh.lianjia.com/zufang/SH216362156310136...,\n 整租·翠庭 2室2厅 南,长宁,虹桥,翠庭
3,http://sh.lianjia.com/zufang/SH224623160808257...,\n 整租·古北嘉年华庭 3室2厅 南,长宁,古北,古北嘉年华庭
4,http://sh.lianjia.com/zufang/SH201375709751857...,\n 整租·西郊华城 5室3厅 复式 南,青浦,徐泾,西郊华城
...,...,...,...,...,...
295,http://sh.lianjia.com/zufang/SH273995825793336...,\n 整租·沿海丽水馨庭(别墅) 5室2厅 南,松江,莘闵别墅,沿海丽水馨庭(别墅)
296,http://sh.lianjia.com/zufang/SH274193324412686...,\n 整租·高兴花园 2室1厅 南,闵行,春申,高兴花园
297,http://sh.lianjia.com/zufang/SH274066724754581...,\n 整租·合生江湾国际公寓 2室2厅 南,杨浦,新江湾城,合生江湾国际公寓
298,http://sh.lianjia.com/zufang/SH274134492568906...,\n 整租·东鼎名人府邸(松江) 2室1厅 南,松江,松江新城,东鼎名人府邸(松江)


In [10]:
import requests

In [11]:
def download_and_parse_web(web):   
    req=Request(web,headers={'User-Agent':'Mozilla/5.0'})   
    downloaded_web=urlopen(req).read()
    soup=BeautifulSoup(downloaded_web)   
    return soup  

In [12]:
import re
def detail_information(soup):    #获取每个房源的详细信息
    a=[]
    b= soup.find('i',attrs={'class':'house_code'}).text  

    a.append(b)
    b= soup.find('p',attrs={'class':'content__title'}).text.strip().split(' ')[0]
    a.append(b)
    b= soup.find('p',attrs={'class':'content__title'}).text.strip().split(' ')[1]
    a.append(b)
    
    b= soup.find('div',attrs={'class':'content__aside--title'}).find('span').text.strip()
    a.append(b)
    
    try:
        b = re.findall( r'\(.*\)',soup.find('div',attrs={'class':'content__aside--title'}).get_text())[0]
    except:
        a.append('无信息')
    else:
        a.append(b)
 
    date_next = soup.find(string='租赁方式：')
    b = date_next.parent.parent.get_text().split('：')[1]
    a.append(b)
    
    b = soup.find('li',string=re.compile("面积")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("朝向")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("维护")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("楼层")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("车位")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("租期")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("入住")).get_text().split('：')[1]
    a.append(b)
    b = soup.find('li',string=re.compile("电梯")).get_text().split('：')[1]
    a.append(b)
    
    try:
        b = soup.find('p',attrs={'data-el':"houseComment"})['data-desc']
    except:
        a.append('无信息')
    else:
        a.append(b)
    
    try:
        b = soup.find('span',string=re.compile("号线")).get_text()  
    except:
        a.append('无信息')
    else:
        a.append(b)
        
    try:
        b = soup.find('span',string=re.compile("号线")).parent.get_text().strip().split('\n')[2]  
    except:
        a.append('无信息')
    else:
        a.append(b)
    
    return a

In [14]:
##尝试一个网站
soup=download_and_parse_web(web['网站'][100])
temp=detail_information(soup)
temp.append(web['网站'][5])
temp

['房源编号：SH2782398623730900992',
 '整租·银都一村',
 '1室1厅',
 '3800',
 '(季付价)',
 '整租',
 '39㎡',
 '南',
 '今天',
 '高楼层/6层',
 '暂无数据',
 '2年以内',
 '随时入住',
 '无',
 '无信息',
 '无信息',
 '无信息',
 'http://sh.lianjia.com/zufang/SH2199687088054468608.html']

In [15]:
room_info=pd.DataFrame(columns=['房源编号','名称1','户型','月租','付款方式','租赁方式','面积','朝向','维护'
                                ,"楼层","车位",'租期','入住','电梯','描述','地铁站','离地铁站距离','网站'])

i=0
s_time = time.time()
for x in web['网站'][0:100]:       #爬取前100个网站的信息 
    soup=download_and_parse_web(x)
    i=i+1
    temp=detail_information(soup)
    temp.append(x)
    room_info.loc[i] =temp
    print('第'+str(i)+'个网站爬取成功')
e_time = time.time()
print('the program runs for {} seconds'.format(e_time-s_time))

第1个网站爬取成功
第2个网站爬取成功
第3个网站爬取成功
第4个网站爬取成功
第5个网站爬取成功
第6个网站爬取成功
第7个网站爬取成功
第8个网站爬取成功
第9个网站爬取成功
第10个网站爬取成功
第11个网站爬取成功
第12个网站爬取成功
第13个网站爬取成功
第14个网站爬取成功
第15个网站爬取成功
第16个网站爬取成功
第17个网站爬取成功
第18个网站爬取成功
第19个网站爬取成功
第20个网站爬取成功
第21个网站爬取成功
第22个网站爬取成功
第23个网站爬取成功
第24个网站爬取成功
第25个网站爬取成功
第26个网站爬取成功
第27个网站爬取成功
第28个网站爬取成功
第29个网站爬取成功
第30个网站爬取成功
第31个网站爬取成功
第32个网站爬取成功
第33个网站爬取成功
第34个网站爬取成功
第35个网站爬取成功
第36个网站爬取成功
第37个网站爬取成功
第38个网站爬取成功
第39个网站爬取成功
第40个网站爬取成功
第41个网站爬取成功
第42个网站爬取成功
第43个网站爬取成功
第44个网站爬取成功
第45个网站爬取成功
第46个网站爬取成功
第47个网站爬取成功
第48个网站爬取成功
第49个网站爬取成功
第50个网站爬取成功
第51个网站爬取成功
第52个网站爬取成功
第53个网站爬取成功
第54个网站爬取成功
第55个网站爬取成功
第56个网站爬取成功
第57个网站爬取成功
第58个网站爬取成功
第59个网站爬取成功
第60个网站爬取成功
第61个网站爬取成功
第62个网站爬取成功
第63个网站爬取成功
第64个网站爬取成功
第65个网站爬取成功
第66个网站爬取成功
第67个网站爬取成功
第68个网站爬取成功
第69个网站爬取成功
第70个网站爬取成功
第71个网站爬取成功
第72个网站爬取成功
第73个网站爬取成功
第74个网站爬取成功
第75个网站爬取成功
第76个网站爬取成功
第77个网站爬取成功
第78个网站爬取成功
第79个网站爬取成功
第80个网站爬取成功
第81个网站爬取成功
第82个网站爬取成功
第83个网站爬取成功
第84个网站爬取成功
第85个网站爬取成功
第86个网站爬取成功
第87个网站爬取成功
第88个网站爬取成功
第89个网站爬取成功
第90个网站爬取成功
第91个网站爬取成功
第92个网站爬取

In [16]:
room_info

,房源编号,名称1,户型,月租,付款方式,租赁方式,面积,朝向,维护,楼层,车位,租期,入住,电梯,描述,地铁站,离地铁站距离,网站
1,房源编号：SH2781717703893983232,整租·双秀西园,2室2厅,4700,(季付价),整租,86㎡,南 北,今天,高楼层/6层,暂无数据,暂无数据,随时入住,无,【房源亮点】<br />\n不临马路，没有噪音，很安静，采光充足，全明户型，2室2厅1厨1卫...,无信息,无信息,http://sh.lianjia.com/zufang/SH278171770389398...
2,房源编号：SH2781996918174121984,整租·高兴花园,1室1厅,3700,(季付价),整租,53㎡,南,今天,低楼层/6层,暂无数据,暂无数据,随时入住,无,无信息,无信息,无信息,http://sh.lianjia.com/zufang/SH278199691817412...
3,房源编号：SH2163621563101364224,整租·翠庭,2室2厅,19000,无信息,整租,169㎡,南,6天前,低楼层/20层,暂无数据,暂无数据,随时入住,有,【房源亮点】<br />\n1.你听说过翠庭精装修的房子有三室出租吗？如果有，那三室租金多少...,"2号线,15号线 - 娄山关路",623m,http://sh.lianjia.com/zufang/SH216362156310136...
4,房源编号：SH2246231608082571264,整租·古北嘉年华庭,3室2厅,26000,(双月付价),整租,144㎡,南,5天前,低楼层/12层,免费使用,1年以内,随时入住,有,【房源亮点】<br />\n小区是2005年建成的，小区的绿植面积很高，小区属于人车分流。下...,"10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 伊犁路",819m,http://sh.lianjia.com/zufang/SH224623160808257...
5,房源编号：SH2013757097518571520,整租·西郊华城,5室3厅,35000,(季付价),整租,266㎡,南,1天前,低楼层/2层,免费使用,3年以内,随时入住,无,【出租原因】<br />\n房屋空置变现，房屋现状保养较好，花园气氛和环境适宜一家人居住<b...,无信息,无信息,http://sh.lianjia.com/zufang/SH201375709751857...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,房源编号：SH2782008083839459328,整租·大西小区,1室1厅,4800,无信息,整租,31㎡,南,1天前,高楼层/6层,暂无数据,暂无数据,随时入住,无,无信息,"8号线,10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 四平路",452m,http://sh.lianjia.com/zufang/SH278200808383945...
97,房源编号：SH2781999667205120000,整租·高兴花园,1室1厅,3300,(季付价),整租,47㎡,南,今天,中楼层/6层,暂无数据,1年以内,随时入住,无,【交通出行】<br />\n高兴花园旁边就有一个菜市场，买菜很方便；出小区在高兴路伟业路那里...,无信息,无信息,http://sh.lianjia.com/zufang/SH278199966720512...
98,房源编号：SH2782003395110903808,整租·虹桥路967弄,1室1厅,7500,(季付价),整租,47㎡,南,1天前,低楼层/6层,暂无数据,1年以内,2021-06-01,无,无信息,"3号线,4号线,10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 虹桥路",419m,http://sh.lianjia.com/zufang/SH278200339511090...
99,房源编号：SH2781950702254030848,整租·创世纪花园,3室2厅,13000,(季付价),整租,124㎡,南,1天前,高楼层/13层,暂无数据,暂无数据,随时入住,有,【交通出行】<br />\n向西走300米1号线4号线上海体育馆地铁站，出门就是公交站，往东...,"1号线,4号线 - 上海体育馆",375m,http://sh.lianjia.com/zufang/SH278195070225403...


In [20]:
room_info1=room_info.reset_index()
room_info1=room_info1.drop('index',axis=1)

In [21]:
room_info1=pd.concat([room_info1,web.iloc[0:100,1:]],axis=1)   #爬取了100个网站
room_info1

,房源编号,名称1,户型,月租,付款方式,租赁方式,面积,朝向,维护,楼层,...,入住,电梯,描述,地铁站,离地铁站距离,网站,名称,区,街道,小区
0,房源编号：SH2781717703893983232,整租·双秀西园,2室2厅,4700,(季付价),整租,86㎡,南 北,今天,高楼层/6层,...,随时入住,无,【房源亮点】<br />\n不临马路，没有噪音，很安静，采光充足，全明户型，2室2厅1厨1卫...,无信息,无信息,http://sh.lianjia.com/zufang/SH278171770389398...,\n 整租·双秀西园 2室2厅 南/北,浦东,康桥,双秀西园
1,房源编号：SH2781996918174121984,整租·高兴花园,1室1厅,3700,(季付价),整租,53㎡,南,今天,低楼层/6层,...,随时入住,无,无信息,无信息,无信息,http://sh.lianjia.com/zufang/SH278199691817412...,\n 整租·高兴花园 1室1厅 南,闵行,春申,高兴花园
2,房源编号：SH2163621563101364224,整租·翠庭,2室2厅,19000,无信息,整租,169㎡,南,6天前,低楼层/20层,...,随时入住,有,【房源亮点】<br />\n1.你听说过翠庭精装修的房子有三室出租吗？如果有，那三室租金多少...,"2号线,15号线 - 娄山关路",623m,http://sh.lianjia.com/zufang/SH216362156310136...,\n 整租·翠庭 2室2厅 南,长宁,虹桥,翠庭
3,房源编号：SH2246231608082571264,整租·古北嘉年华庭,3室2厅,26000,(双月付价),整租,144㎡,南,5天前,低楼层/12层,...,随时入住,有,【房源亮点】<br />\n小区是2005年建成的，小区的绿植面积很高，小区属于人车分流。下...,"10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 伊犁路",819m,http://sh.lianjia.com/zufang/SH224623160808257...,\n 整租·古北嘉年华庭 3室2厅 南,长宁,古北,古北嘉年华庭
4,房源编号：SH2013757097518571520,整租·西郊华城,5室3厅,35000,(季付价),整租,266㎡,南,1天前,低楼层/2层,...,随时入住,无,【出租原因】<br />\n房屋空置变现，房屋现状保养较好，花园气氛和环境适宜一家人居住<b...,无信息,无信息,http://sh.lianjia.com/zufang/SH201375709751857...,\n 整租·西郊华城 5室3厅 复式 南,青浦,徐泾,西郊华城
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,房源编号：SH2782008083839459328,整租·大西小区,1室1厅,4800,无信息,整租,31㎡,南,1天前,高楼层/6层,...,随时入住,无,无信息,"8号线,10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 四平路",452m,http://sh.lianjia.com/zufang/SH278200808383945...,\n 整租·大西小区 1室1厅 南,虹口,曲阳,大西小区
96,房源编号：SH2781999667205120000,整租·高兴花园,1室1厅,3300,(季付价),整租,47㎡,南,今天,中楼层/6层,...,随时入住,无,【交通出行】<br />\n高兴花园旁边就有一个菜市场，买菜很方便；出小区在高兴路伟业路那里...,无信息,无信息,http://sh.lianjia.com/zufang/SH278199966720512...,\n 整租·高兴花园 1室1厅 南,闵行,春申,高兴花园
97,房源编号：SH2782003395110903808,整租·虹桥路967弄,1室1厅,7500,(季付价),整租,47㎡,南,1天前,低楼层/6层,...,2021-06-01,无,无信息,"3号线,4号线,10号线(新江湾城-虹桥火车站),10号线(新江湾城-航中路) - 虹桥路",419m,http://sh.lianjia.com/zufang/SH278200339511090...,\n 整租·虹桥路967弄 1室1厅 南,长宁,新华路,虹桥路967弄
98,房源编号：SH2781950702254030848,整租·创世纪花园,3室2厅,13000,(季付价),整租,124㎡,南,1天前,高楼层/13层,...,随时入住,有,【交通出行】<br />\n向西走300米1号线4号线上海体育馆地铁站，出门就是公交站，往东...,"1号线,4号线 - 上海体育馆",375m,http://sh.lianjia.com/zufang/SH278195070225403...,\n 整租·创世纪花园 3室2厅 南,徐汇,徐家汇,创世纪花园


In [38]:
room_info1.to_csv('链家房源信息.csv',index=False,encoding='utf-8-sig')